In [11]:
# ==============================================================================
# 1. IMPORTAÇÕES E CONFIGURAÇÕES GERAIS
# ==============================================================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import shap
import os
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Configuração de Estilo Global
sns.set_context("notebook", font_scale=1.1)
sns.set_style("whitegrid")

In [12]:
# ==============================================================================
# 2. CONFIGURAÇÃO DO PROJETO (CONSTANTES)
# ==============================================================================
class Config:
    """Centraliza constantes, caminhos e configurações do projeto."""
    
    # Caminhos
    BASE_DIR = Path.cwd()
    DATA_PATH = BASE_DIR.parent / 'data' / 'obesity.csv'
    OUTPUT_MODEL_DIR = BASE_DIR.parent / 'saved_model'
    OUTPUT_DOCS_DIR = BASE_DIR.parent / 'docs' / 'assets'
    
    # Colunas
    TARGET_COL = 'Obesity'
    COLS_TO_ROUND = ['FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
    
    # Definição de Features
    NUMERIC_FEATURES = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
    ONE_HOT_FEATURES = ['Gender', 'family_history', 'FAVC', 'SMOKE', 'SCC', 'MTRANS']
    ORDINAL_FEATURES = ['CAEC', 'CALC']
    
    # Ordem das Categorias Ordinais
    ORDER_CAEC = ['no', 'Sometimes', 'Frequently', 'Always']
    ORDER_CALC = ['no', 'Sometimes', 'Frequently', 'Always']
    
    # Dicionários de Tradução (PT-BR)
    TRANSLATE_CLASSES = {
        'Insufficient_Weight': 'Abaixo do Peso',
        'Normal_Weight': 'Peso Normal',
        'Overweight_Level_I': 'Sobrepeso I',
        'Overweight_Level_II': 'Sobrepeso II',
        'Obesity_Type_I': 'Obesidade I',
        'Obesity_Type_II': 'Obesidade II',
        'Obesity_Type_III': 'Obesidade III'
    }
    
    TRANSLATE_FEATURES = {
        'Age': 'Idade', 'Height': 'Altura', 'Weight': 'Peso',
        'family_history': 'Histórico Familiar', 'FAVC': 'Consumo Calórico',
        'FCVC': 'Consumo de Vegetais', 'NCP': 'Refeições/Dia',
        'CAEC': 'Comer entre Refeições', 'SMOKE': 'Fumante',
        'CH2O': 'Consumo de Água', 'SCC': 'Monitora Calorias',
        'FAF': 'Atividade Física', 'TUE': 'Tempo de Tela',
        'CALC': 'Consumo de Álcool', 'MTRANS': 'Transporte',
        'Gender': 'Gênero'
    }
    
    TRANSLATE_VALUES = {
        'yes': 'Sim', 'no': 'Não',
        'Male': 'Masc.', 'Female': 'Fem.',
        'Automobile': 'Carro', 'Public_Transportation': 'Transp. Público',
        'Walking': 'Caminhada', 'Motorbike': 'Moto', 'Bike': 'Bicicleta',
        'Sometimes': 'Às vezes', 'Frequently': 'Freq.', 'Always': 'Sempre'
    }

# Garante que diretórios de saída existem
Config.OUTPUT_MODEL_DIR.mkdir(parents=True, exist_ok=True)
Config.OUTPUT_DOCS_DIR.mkdir(parents=True, exist_ok=True)

In [13]:
# ==============================================================================
# 3. FUNÇÕES AUXILIARES (CORE)
# ==============================================================================

def carregar_e_limpar_dados(caminho_arquivo: Path) -> pd.DataFrame:
    """
    Carrega o dataset e aplica limpezas iniciais (arredondamento).
    
    Args:
        caminho_arquivo (Path): Caminho para o CSV.
        
    Returns:
        pd.DataFrame: DataFrame limpo.
    """
    try:
        df = pd.read_csv(caminho_arquivo)
    except FileNotFoundError:
        # Tenta carregar do diretório atual como fallback
        print(f"Arquivo não encontrado em {caminho_arquivo}. Tentando local...")
        df = pd.read_csv('obesity.csv')
        
    # Arredonda colunas numéricas que possuem ruído decimal
    for col in Config.COLS_TO_ROUND:
        if col in df.columns:
            df[col] = df[col].round().astype(int)
            
    print(f"Dados carregados. Shape: {df.shape}")
    return df

def construir_pipeline() -> Pipeline:
    """
    Constrói o pipeline de pré-processamento e modelagem.
    
    
    Returns:
        Pipeline: Objeto Scikit-Learn pronto para treino.
    """
    # 1. Transformadores Específicos
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    ordinal_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal_encoder', OrdinalEncoder(
            categories=[Config.ORDER_CAEC, Config.ORDER_CALC],
            handle_unknown='use_encoded_value',
            unknown_value=-1
        ))
    ])

    one_hot_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # 2. Pré-processador Geral (ColumnTransformer)
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, Config.NUMERIC_FEATURES),
            ('cat_onehot', one_hot_transformer, Config.ONE_HOT_FEATURES),
            ('cat_ordinal', ordinal_transformer, Config.ORDINAL_FEATURES)
        ],
        remainder='passthrough'
    )

    # 3. Pipeline Final com Random Forest
    return Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
    ])

def limpar_nome_feature(nome_sujo: str) -> str:
    """
    Traduz e limpa nomes técnicos de features gerados pelo Pipeline.
    Ex: 'cat_onehot__Gender_Male' -> 'Gênero: Masc.'
    """
    # Remove prefixos técnicos (ex: cat_onehot__)
    nome_limpo = nome_sujo.split('__')[-1]
    
    # Tradução direta
    if nome_limpo in Config.TRANSLATE_FEATURES:
        return Config.TRANSLATE_FEATURES[nome_limpo]
    
    # Tradução composta (Feature + Valor OneHot)
    for feature_eng, feature_pt in Config.TRANSLATE_FEATURES.items():
        if feature_eng in nome_limpo:
            valor_eng = nome_limpo.replace(f"{feature_eng}_", "")
            valor_pt = Config.TRANSLATE_VALUES.get(valor_eng, valor_eng)
            return f"{feature_pt}: {valor_pt}"
            
    return nome_limpo

def salvar_modelo(pipeline, X_train):
    """Salva o modelo treinado e seus metadados essenciais."""
    
    # Salva Pipeline
    path_model = Config.OUTPUT_MODEL_DIR / 'modelo_obesidade.joblib'
    joblib.dump(pipeline, path_model)
    
    # Salva Metadados
    model_metadata = {
        'features_expected': X_train.columns.tolist(),
        'numeric_features': Config.NUMERIC_FEATURES,
        'one_hot_features': Config.ONE_HOT_FEATURES,
        'ordinal_features': Config.ORDINAL_FEATURES,
        'classes': pipeline.classes_.tolist()
    }
    path_meta = Config.OUTPUT_MODEL_DIR / 'model_metadata.joblib'
    joblib.dump(model_metadata, path_meta)
    
    print(f"\nModelo salvo em: {path_model}")
    print(f"Metadados salvos em: {path_meta}")

In [14]:
# ==============================================================================
# 4. FUNÇÕES DE VISUALIZAÇÃO (DOCUMENTAÇÃO)
# ==============================================================================

def plotar_matriz_confusao(y_test, y_pred, classes, output_dir):
    """Gera e salva a Matriz de Confusão traduzida."""
    labels_traduzidos = [Config.TRANSLATE_CLASSES.get(l, l) for l in classes]
    cm = confusion_matrix(y_test, y_pred, labels=classes)

    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm, annot=True, fmt='d', cmap='Blues',
        xticklabels=labels_traduzidos, yticklabels=labels_traduzidos, cbar=False
    )
    plt.title('Matriz de Confusão (Dados de Teste)', fontsize=16, pad=20)
    plt.xlabel('Predição do Modelo', fontsize=12)
    plt.ylabel('Classificação Real', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(output_dir / 'matriz_confusao.png', dpi=300)
    plt.close()
    print("Imagem salva: matriz_confusao.png")

def plotar_feature_importance(pipeline, output_dir):
    """Extrai, limpa e plota a importância das features."""
    # Recupera nomes das features transformadas
    preprocessor = pipeline.named_steps['preprocessor']
    raw_names = []
    
    # Numéricas
    raw_names.extend(Config.NUMERIC_FEATURES)
    # OneHot
    ohe = preprocessor.named_transformers_['cat_onehot']['onehot']
    raw_names.extend(ohe.get_feature_names_out(Config.ONE_HOT_FEATURES))
    # Ordinais
    raw_names.extend(Config.ORDINAL_FEATURES)
    
    # Limpa nomes e pega importâncias
    clean_names = [limpar_nome_feature(f) for f in raw_names]
    importances = pipeline.named_steps['classifier'].feature_importances_
    
    # DataFrame para plotagem
    df_imp = pd.DataFrame({'Feature': clean_names, 'Importance': importances})
    df_imp = df_imp.groupby('Feature').sum().reset_index() # Agrupa duplicatas se houver
    df_imp = df_imp.sort_values('Importance', ascending=False).head(15)

    plt.figure(figsize=(12, 7))
    ax = sns.barplot(x='Importance', y='Feature', data=df_imp, palette='viridis', hue='Feature', legend=False)
    plt.title('Fatores de Maior Impacto na Predição', fontsize=16)
    
    for i in ax.containers:
        ax.bar_label(i, fmt='%.3f', padding=5)
        
    sns.despine(left=True, bottom=False)
    plt.tight_layout()
    plt.savefig(output_dir / 'feature_importance.png', dpi=300)
    plt.close()
    print("Imagem salva: feature_importance.png")

def plotar_shap_waterfall(pipeline, X_test, output_dir):
    """Gera um gráfico Waterfall do SHAP para um caso específico (Obesidade III)."""
    # Configura estilo específico para SHAP
    plt.style.use('default') 
    
    class_interest = 'Obesity_Type_III'
    if class_interest not in pipeline.classes_:
        return

    class_idx = list(pipeline.classes_).index(class_interest)
    preds = pipeline.predict(X_test)
    indices = np.where(preds == class_interest)[0]

    if len(indices) == 0:
        print("Nenhum caso de Obesidade III encontrado para gerar SHAP.")
        return

    # Prepara dados
    sample_idx = indices[0]
    X_sample = X_test.iloc[[sample_idx]]
    X_trans = pipeline.named_steps['preprocessor'].transform(X_sample)

    # Calcula SHAP
    model = pipeline.named_steps['classifier']
    explainer = shap.TreeExplainer(model)
    shap_values = explainer(X_trans)
    explanation = shap_values[0, :, class_idx]

    # Traduz nomes
    preprocessor = pipeline.named_steps['preprocessor']
    raw_names = (Config.NUMERIC_FEATURES + 
                 list(preprocessor.named_transformers_['cat_onehot']['onehot'].get_feature_names_out(Config.ONE_HOT_FEATURES)) + 
                 Config.ORDINAL_FEATURES)
    explanation.feature_names = [limpar_nome_feature(f) for f in raw_names]

    plt.figure(figsize=(10, 6))
    shap.plots.waterfall(explanation, max_display=12, show=False)
    plt.title(f"Explicação: {Config.TRANSLATE_CLASSES.get(class_interest, class_interest)}", fontsize=14)
    plt.tight_layout()
    plt.savefig(output_dir / 'shap_waterfall.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Restaura estilo
    sns.set_style("whitegrid")
    print("Imagem salva: shap_waterfall.png")

In [15]:
# ==============================================================================
# 5. EXECUÇÃO PRINCIPAL
# ==============================================================================

if __name__ == "__main__":
    
    # 1. Carregamento
    df = carregar_e_limpar_dados(Config.DATA_PATH)
    
    # 2. Split
    X = df.drop(Config.TARGET_COL, axis=1)
    y = df[Config.TARGET_COL]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # 3. Treinamento
    print("\nIniciando treinamento do Pipeline...")
    model_pipeline = construir_pipeline()
    model_pipeline.fit(X_train, y_train)
    print("Modelo treinado com sucesso.")
    
    # 4. Avaliação
    acc = model_pipeline.score(X_test, y_test)
    print(f"\nAcurácia no Teste: {acc:.2%}")
    
    # Cross Validation (rápido check)
    cv_scores = cross_val_score(model_pipeline, X, y, cv=5, scoring='accuracy')
    print(f"Cross-Validation (Média): {cv_scores.mean():.2%} (+/- {cv_scores.std():.2%})")
    
    # 5. Persistência
    salvar_modelo(model_pipeline, X_train)
    
    # 6. Geração de Documentação Visual
    print("\nGerando assets para documentação...")
    y_pred = model_pipeline.predict(X_test)
    
    plotar_matriz_confusao(y_test, y_pred, model_pipeline.classes_, Config.OUTPUT_DOCS_DIR)
    plotar_feature_importance(model_pipeline, Config.OUTPUT_DOCS_DIR)
    plotar_shap_waterfall(model_pipeline, X_test, Config.OUTPUT_DOCS_DIR)
    
    print("\nProcesso finalizado com sucesso!")

Dados carregados. Shape: (2111, 17)

Iniciando treinamento do Pipeline...
Modelo treinado com sucesso.

Acurácia no Teste: 94.09%
Cross-Validation (Média): 92.24% (+/- 10.44%)

Modelo salvo em: c:\Users\Pedro\Desktop\Git\obesity\saved_model\modelo_obesidade.joblib
Metadados salvos em: c:\Users\Pedro\Desktop\Git\obesity\saved_model\model_metadata.joblib

Gerando assets para documentação...
Imagem salva: matriz_confusao.png
Imagem salva: feature_importance.png
Imagem salva: shap_waterfall.png

Processo finalizado com sucesso!
